In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# River Distance

I've written a couple of Python modules to extend the capabilities of `networkx.DiGraph` to help with navigating river networks in the NHD. This notebook will demonstrate their use in building a network graph, and finding the shortest river path to the ocean from a given point.

In [2]:
from graph_prep import GraphBuilder
from river_graph import RiverGraph

## Preparing a River Graph from an NHD Shapefile

First, read the shapefile into a `graph_prep.GraphBuilder` object.

In [3]:
%%time
gb = GraphBuilder('data/kusko_flowlines.shp')

CPU times: user 32.7 s, sys: 1.32 s, total: 34 s
Wall time: 34.1 s


This is what that looks like:

<img src="images/all_HUC_lines.png" width="400px" />

Prune off the parts of the network that are not connected to the coastline. This is done by breaking the graph into weakly connected component subgraphs, and only keeping the subgraphs which contain coastal nodes. Coastal nodes are nodes connected to at least one edge that is attributed with the [NHD FCode for coastline](https://nhd.usgs.gov/userGuide/Robohelpfiles/NHD_User_Guide/Feature_Catalog/Hydrography_Dataset/Complete_FCode_List.htm) ($56600$).

In [4]:
%%time
gb.graph = gb.prune_network(True)

47 graphs with coastal nodes, 4898 without.
CPU times: user 3min 13s, sys: 6.7 s, total: 3min 20s
Wall time: 3min 20s


Now with fewer rivers:

<img src="images/pruned_HUC_lines.png" width="400px" />

Now save the graph so that it can be loaded again without having to convert the shapefile and do the pruning. 

In [6]:
# gb.write_gpickle('data/kusko_pruned.pkl')

Loading pickles doesn't seem to be working. The loaded graph is behaving as though it were not a directed graph. More testing required.

~~The saved file can now be loaded using the `GraphBuilder` class:~~ 

```python
gb = GraphBuilder('data/kusko_pruned.pkl')
```

<sub>Note: If you're using a customized application with a different FCode, you'll have to set that FCode again when you load the object.</sub>

## Using the RiverGraph to find paths

First, I'll load a shapefile of test start points.

In [7]:
import geopandas as gpd

In [8]:
starts = gpd.read_file('data/test_points.shp')

Here are the test points on the map:

<img src="images/test_points.png" width="400px" />

Getting coordinates as a tuple from a Shapely point geometry is a bit awkward.

In [56]:
g = starts.geometry[0]
tuple(np.array(g.coords)[0])

(-320315.0, 1284942.0)

This very quickly calculates the path geometries for the points in the test shapefile.

In [50]:
%%time
paths = starts.geometry.apply(lambda g: gb.graph.shortest_path_to_coast(
                                   gb.graph.closest_node(tuple(np.array(g.coords)[0]))))

CPU times: user 2.21 s, sys: 9.84 ms, total: 2.22 s
Wall time: 2.22 s


Print the lengths for the paths in km.

In [57]:
for leng in [p.length * .001 for p in paths]:
    print "{:.2f} km".format(leng)

294.15 km
18.95 km
65.69 km
240.46 km
123.90 km
255.01 km
95.74 km
91.98 km
299.18 km


Puth the paths in a GeoDataFrame and save to a shapefile.

In [61]:
pgdf = gpd.geodataframe.GeoDataFrame(paths)
pgdf.crs = starts.crs
pgdf.to_file('data/test_paths.shp')

Here are the paths to the coast from the test points:

<img src="images/test_output.png" width="600px" />

Now I just need to scale this up to the whole state.